[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NirDiamant/RAG_Techniques/blob/main/all_rag_techniques/HyDe_Hypothetical_Document_Embedding.ipynb)

# 文档检索中的假设性文档嵌入 (HyDE)

## 概述

此代码实现了一个用于文档检索的假设性文档嵌入 (HyDE) 系统。HyDE 是一种创新方法，它将查询问题转换为包含答案的假设性文档，旨在弥合向量空间中查询和文档分布之间的差距。

## 动机

传统的检索方法通常难以解决短查询与更长、更详细的文档之间的语义差距。HyDE 通过将查询扩展为一个完整的假设性文档来解决这个问题，通过使查询表示在向量空间中与文档表示更相似，从而可能提高检索相关性。

## 关键组件

1. PDF 处理和文本分块
2. 使用 FAISS 和 OpenAI 嵌入创建向量存储
3. 用于生成假设性文档的语言模型
4. 实现 HyDE 技术的自定义 HyDERetriever 类

## 方法详情

### 文档预处理和向量存储创建

1. 处理 PDF 并将其拆分为块。
2. 使用 OpenAI 嵌入创建 FAISS 向量存储，以实现高效的相似性搜索。

### 假设性文档生成

1. 使用语言模型 (GPT-4) 生成一个回答给定查询的假设性文档。
2. 生成过程由一个提示模板引导，该模板确保假设性文档详细并与向量存储中使用的块大小相匹配。

### 检索过程

`HyDERetriever` 类实现以下步骤：

1. 使用语言模型从查询中生成一个假设性文档。
2. 将假设性文档用作向量存储中的搜索查询。
3. 检索与此假设性文档最相似的文档。

## 主要特点

1. 查询扩展：将短查询转换为详细的假设性文档。
2. 灵活配置：允许调整块大小、重叠和检索到的文档数量。
3. 与 OpenAI 模型集成：使用 GPT-4 进行假设性文档生成，并使用 OpenAI 嵌入进行向量表示。

## 这种方法的好处

1. 提高相关性：通过将查询扩展为完整文档，HyDE 可以潜在地捕获更细致和相关的匹配。
2. 处理复杂查询：对于可能难以直接匹配的复杂或多方面查询特别有用。
3. 适应性：假设性文档生成可以适应不同类型的查询和文档领域。
4. 更好地理解上下文的潜力：扩展后的查询可能更好地捕捉原始问题背后的上下文和意图。

## 实现细节

1. 使用 OpenAI 的 ChatGPT 模型进行假设性文档生成。
2. 使用 FAISS 在向量空间中进行高效的相似性搜索。
3. 允许轻松可视化假设性文档和检索到的结果。

## 结论

假设性文档嵌入 (HyDE) 代表了一种创新的文档检索方法，解决了查询和文档之间的语义差距。通过利用先进的语言模型将查询扩展为假设性文档，HyDE 有可能显著提高检索相关性，特别是对于复杂或细致的查询。这种技术在理解查询意图和上下文至关重要的领域可能特别有价值，例如法律研究、学术文献综述或高级信息检索系统。

<div style="text-align: center;">

<img src="../images/HyDe.svg" alt="HyDe" style="width:40%; height:auto;">
</div>

<div style="text-align: center;">

<img src="../images/hyde-advantages.svg" alt="HyDe" style="width:100%; height:auto;">
</div>

# 包安装和导入

下面的单元格安装了运行此笔记本所需的所有必要软件包。


In [ ]:
# Install required packages
!pip install python-dotenv

In [ ]:
# Clone the repository to access helper functions and evaluation modules
!git clone https://github.com/NirDiamant/RAG_TECHNIQUES.git
import sys
sys.path.append('RAG_TECHNIQUES')
# If you need to run with the latest data
# !cp -r RAG_TECHNIQUES/data .

In [ ]:
import os
import sys
from dotenv import load_dotenv


# Original path append replaced for Colab compatibility
from helper_functions import *
from evaluation.evalute_rag import *

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### 定义文档路径

In [ ]:
# Download required data files
import os
os.makedirs('data', exist_ok=True)

# Download the PDF document used in this notebook
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf


In [2]:
path = "data/Understanding_Climate_Change.pdf"

### 定义 HyDe 检索器类 - 创建向量存储、生成假设文档和检索

In [3]:
class HyDERetriever:
    def __init__(self, files_path, chunk_size=500, chunk_overlap=100):
        self.llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini", max_tokens=4000)

        self.embeddings = OpenAIEmbeddings()
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.vectorstore = encode_pdf(files_path, chunk_size=self.chunk_size, chunk_overlap=self.chunk_overlap)
    
        
        self.hyde_prompt = PromptTemplate(
            input_variables=["query", "chunk_size"],
            template="""Given the question '{query}', generate a hypothetical document that directly answers this question. The document should be detailed and in-depth.
            the document size has be exactly {chunk_size} characters.""",
        )
        self.hyde_chain = self.hyde_prompt | self.llm

    def generate_hypothetical_document(self, query):
        input_variables = {"query": query, "chunk_size": self.chunk_size}
        return self.hyde_chain.invoke(input_variables).content

    def retrieve(self, query, k=3):
        hypothetical_doc = self.generate_hypothetical_document(query)
        similar_docs = self.vectorstore.similarity_search(hypothetical_doc, k=k)
        return similar_docs, hypothetical_doc


### Create a HyDe retriever instance

In [4]:
retriever = HyDERetriever(path)

### Demonstrate on a use case

In [5]:
test_query = "What is the main cause of climate change?"
results, hypothetical_doc = retriever.retrieve(test_query)

### Plot the hypothetical document and the retrieved documnets 

In [ ]:
docs_content = [doc.page_content for doc in results]

print("hypothetical_doc:\n")
print(text_wrap(hypothetical_doc)+"\n")
show_context(docs_content)